## Loading Libraries and establishing connection to the database

In [1]:
import pymysql
import pandas as pd
import yfinance as yf

conn = pymysql.connect(host = '130.211.219.71',port=int(3306),user='root',passwd='Nodal@987',db='nodalDB')
cur = conn.cursor()

## Nifty and Banknifty stocks

In [2]:
nifty = pd.read_csv("nifty.csv")
banknifty = pd.read_csv("banknifty.csv")

In [3]:
nifty_list = nifty['Symbol'].tolist()
banknifty_list = banknifty['Symbol'].tolist()

In [4]:
print(nifty_list)
print(len(nifty_list))
print(banknifty_list)
print(len(banknifty_list))
final_list = list(sorted(set(nifty_list) | set(banknifty_list)))
print(final_list)
print(len(final_list))

['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'IOC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SHREECEM', 'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO']
50
['AUBANK', 'AXISBANK', 'BANDHANBNK', 'FEDERALBNK', 'HDFCBANK', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'KOTAKBANK', 'PNB', 'RBLBANK', 'SBIN']
12
['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO'

## Function to add data to the database table

In [ ]:
from time import gmtime, strftime

def add_data(stock_name = "", start_date = "", end_date = ""):
    name = stock_name + ".NS"
    stock = yf.download(name, start=start_date, end=end_date, progress=False,)
    stock.dropna(inplace = True)
    stock.reset_index(inplace = True)
    stock.rename(columns = {'Date':'trading_date', 'Open' : 'open', 'High' : 'high','Low' : 'low','Close' : 'close', 'Adj Close' : 'adj_close', 'Volume' : 'volume'}, inplace = True)
    stock_name = stock_name
    stock['stock_name'] = [stock_name] * stock.shape[0]
    stock['timestamp'] = [strftime("%Y-%m-%d %H:%M:%S", gmtime())] * stock.shape[0]
    
    for i,row in stock.iterrows():
        sql = """INSERT INTO nifty_banknifty_stocks_list(trading_date, open, high, low, close, adj_close, volume, stock_name, timestamp) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cur.execute(sql,tuple(row))
        print("Record Added", i)
    conn.commit()

In [ ]:
for stocks in final_list:
    print("Trying to add {} in the database".format(stocks))
    add_data(stock_name = stocks,start_date = '2015-01-01',end_date = '2021-11-01')
    print(stocks, "is added successfully to the database.")

## Fetching the data from the database table

In [ ]:
df = pd.read_sql_query("SELECT * FROM y_finance_daily ",conn)
df.head()

In [6]:
stock = yf.download("TATAMOTORS.NS", start="2022-01-07", end='2022-01-08', progress=False,)

In [7]:
stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-07,490.049988,495.5,483.75,490.600006,490.600006,15532165
